<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-Parameter" data-toc-modified-id="Set-Parameter-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set Parameter</a></span></li><li><span><a href="#Print-Routine" data-toc-modified-id="Print-Routine-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Print Routine</a></span></li><li><span><a href="#Define-Variables" data-toc-modified-id="Define-Variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Variables</a></span></li><li><span><a href="#Define-Problem" data-toc-modified-id="Define-Problem-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Define Problem</a></span></li><li><span><a href="#Add-Constraints" data-toc-modified-id="Add-Constraints-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add Constraints</a></span></li><li><span><a href="#Generate-solutions" data-toc-modified-id="Generate-solutions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Generate solutions</a></span></li></ul></div>

In [1]:
import pulp

# Set Parameter

In [2]:
n=9

# Print Routine

General Formatting Routine prints a chess board of size `n` with Queens (a bit of Python trickery)

In [3]:
def print_solution(n, q):
    for i in range(0,n):
        for j in range(0,n):
            print('+-', end='')
        else:
            print('+')
        for j in range(0,n):
            if pulp.value(q[i][j])==1:
                print('|#', end='')
            else:
                print('| ', end='')
        else:
            print('|')
    else:
        for j in range(0,n):
            print('+-', end='')
        else:
            print('+')
    print()

# Define Variables

For each row `i` and column `j` generate an integer variable `Q_i_j` with values between 0 and 1:

In [4]:
q=pulp.LpVariable.dicts("Q", (range(0,n), range(0,n)),
                        lowBound=0, upBound=1, cat=pulp.LpInteger)

For example:

In [5]:
q[0][1]

Q_0_1

# Define Problem

Generate the problem: maximise the numbers of queens on the chess board:

In [6]:
prob = pulp.LpProblem("Queens",pulp.LpMaximize)

We know that if there is a solution the maximum would be $n$ but we give the system something to do...

In [7]:
prob += pulp.lpSum([ q[i][j] for i in range(0, n) for j in range(0, n)])

# Add Constraints

For all rows there is one queen in the row

In [8]:
for i in range(0, n):
    prob += pulp.lpSum([ q[i][j] for j in range(0, n) ]) == 1

For all columns there is one queen in the column

In [9]:
for j in range(0, n):
    prob += pulp.lpSum([ q[i][j] for i in range(0, n) ]) == 1

In each diagonal there is at most one queen

In [10]:
for i in range(0,n):
    for j in range(0,n):
        prob += pulp.lpSum([ q[i+k][j+k] 
                                 for k in range(-n, n) 
                                       if 0<=min(i+k, j+k) and max(i+k, j+k)<n]) <= 1

In [11]:
for i in range(0,n):
    for j in range(0,n):
        prob += pulp.lpSum([ q[i+k][j-k] 
                                 for k in range(-n, n) 
                                       if 0<=min(i+k, j-k) and max(i+k, j-k)<n]) <= 1

# Generate solutions

In [12]:
k=0
status=prob.solve()
while status==1:
    k+=1
    print_solution(n, q)
    # add another constraint essentially excluding this solution
    prob += pulp.lpSum([ q[i][j]  
                         for i in range(0, n) for j in range(0, n)
                           if pulp.value(q[i][j])==1
                       ]) <= n-1
    status=prob.solve()
print(f'For n={n:d} there are {k:d} Solutions')

+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | 

+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | 

+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | 

+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | 

+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | 

...

+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | 

+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#

+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | 

+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| |#| | | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | |#| | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | |
+-+-+-+-+-+-+-+-+-+
|#| | | | | | | | |
+-+-+-+-+-+-+-+-+-+

+-+-+-+-+-+-+-+-+-+
| | | | | | | |#| |
+-+-+-+-+-+-+-+-+-+
| | | | |#| | | | |
+-+-+-+-+-+-+-+-+-+
| | |#| | | | | | |
+-+-+-+-+-+-+-+-+-+
| | | | | |#| | | |
+-+-+-+-+-+-+-+-+-+
| | | | | | | | |#|
+-+-+-+-+-+-+-+-+-+
| | | | | | |#| | 

For n=9 there are 352 Solutions
